In [ ]:
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import plotly.express as px
from EC_data_processing_lib import get_header_length, get_decimal_separator
from Data_setup_CHEM import data_set  # choose the correct python file
import os
from galvani import BioLogic
from datetime import datetime

fig_title = 'Title'
today_str = datetime.today().strftime('%Y-%m-%d')
fig = go.Figure()
colors = px.colors.qualitative.Dark24

for i in data_set:
    if not os.path.exists(data_set[i]['data_CV']):
        continue
    if data_set[i]['data_CV'].endswith('mpt'):
        headerlength = get_header_length(data_set[i]['data_CV'])
        data = pd.read_csv(
            data_set[i]['data_CV'],
            encoding='ansi',
            sep="\t",
            decimal=get_decimal_separator(data_set[i]['data_CV']),
            skiprows=range(headerlength)
        )
    else:
        mpr_file = BioLogic.MPRfile(data_set[i]['data_CV'])
        data = pd.DataFrame(mpr_file.data)
    
    # Automatically select the correct current column
    if '<I>/mA' in data.columns:
        current_column = '<I>/mA'
    elif 'I/mA' in data.columns:
        current_column = 'I/mA'
    else:
        continue

    cn = data['cycle number']
    # Plot only the last scan:
    last_cycle = cn.iloc[-1]
    data = data[data['cycle number'] == float(last_cycle)]
    
    voltage = data['Ewe/V']
    current = data[current_column]
    current = current / data_set[i]['mass']  # normalize to mass (optional)
    voltage_corrected = data['Ewe/V'] - data[current_column] * data_set[i]['Ru'] / 1000 - data_set[i]['cal']

    fig.add_trace(go.Scatter(
        x=voltage_corrected,
        y=current,
        name=data_set[i]['label'],
        line_color=colors[data_set[i]['color_index']],
        line_width=3,
    ))

# Set axes to autorange along with additional styling
fig.update_xaxes(
    autorange=True,
    nticks=8,
    ticks='outside',
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    zeroline=False,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

fig.update_yaxes(
    autorange=True,
    nticks=7,
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    ticks='outside',
    zeroline=True,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

fig.update_layout(
    xaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Arial, monospace",
        size=22,
        color="black"
    )
)

# Set layout for saving the SVG: transparent background
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(0, 0, 0, 0)"
},
    width=500,
    height=500,
    legend=dict(
        yanchor="top",
        y=1.04,
        xanchor="left",
        x=0
    ),
    margin={'t': 0, 'l': 0, 'b': 0, 'r': 10},
    legend_font_size=22
)

# Save the SVG (transparent background)
fig.write_image(f"{today_str} {fig_title} Cyclic Voltammetry.svg")

# Update layout for a white background for display and JPEG saving
fig.update_layout({
    "plot_bgcolor": "rgba(255, 255, 255, 1)",
    "paper_bgcolor": "rgba(255, 255, 255, 1)"
})

fig.show()

# Save the JPEG with a white background
fig.write_image(f"{today_str} {fig_title} Cyclic Voltammetry.jpeg")
